# Exploring Network History in NeKo

This notebook demonstrates the new history facilities in NeKo. You will learn how to:

- capture network states automatically while you enrich a network
- inspect branches and metadata associated with each state
- visualise the history graph as both a `networkx` object and an embedded SVG
- control the history recorder with context managers and global toggles
- prune older history entries using `set_max_history`

In [ ]:
import pandas as pd
from IPython.display import display, HTML
from neko.core.network import Network

## 1. Create a network with sample resources
We will use a small Omnipath-like edge set built from Uniprot identifiers.

In [ ]:
sample_resources = pd.DataFrame({
    "source": ["P31749", "Q9Y243", "P46087", "Q9Y243", "P01112"],
    "target": ["P17302", "P31749", "Q15796", "Q9Y6K9", "P84022"],
    "is_directed": [True, True, True, True, True],
    "is_stimulation": [True, False, True, False, True],
    "is_inhibition": [False, True, False, True, False],
    "form_complex": [False, False, False, False, False],
})
sample_resources

In [ ]:
net = Network(initial_nodes=["P31749"], resources=sample_resources)
net.list_states()

## 2. Automatic history capture
Every mutating API call creates a new state with metadata describing the operation.

In [ ]:
net.add_node("Q9Y243")
net.add_node("P46087")
net.list_states()

We can compare branches or move around the history using `checkout`, `undo`, and `redo`.

In [ ]:
current = net.current_state_id
net.undo()
previous = net.current_state_id
net.redo()
current, previous

## 3. Visualising the history graph
`history_graph()` returns a `networkx` object, while `history_html()` embeds a Graphviz SVG.

In [ ]:
history_nx = net.history_graph()
list(history_nx.edges())

In [ ]:
HTML(net.history_html())

## 4. Working with branches
We can revert to an earlier state, explore an alternative enrichment strategy, and compare the resulting states.

In [ ]:
root_state = net.root_state_id
net.checkout(root_state)
net.add_node("P01112")
branch_state = net.current_state_id
diff = net.compare_states(current, branch_state)
diff

## 5. Suspension and toggles
Use `suspend_history()` or `set_history_tracking(False)` to batch changes without recording intermediate states.

In [ ]:
with net.suspend_history():
    net.add_node("Q9Y243")  # already present, but no state recorded
    net.list_states()[-1]


In [ ]:
net.set_history_tracking(False)
net.add_node("Q15796")
net.set_history_tracking(True)
net.list_states()


## 6. Pruning history
`set_max_history()` limits the number of stored states while preserving the root and the latest state.

In [ ]:
net.set_max_history(4)
for node in ["Q9Y6K9", "P17302"]:
    if node not in net.nodes["Genesymbol"].tolist():
        net.add_node(node)
net.list_states()


When the number of stored states exceeds the limit, older leaf states are pruned automatically. The root and current state remain intact.

This concludes the walkthrough of the history API. Experiment by combining these features with your favourite enrichment strategies to analyse multiple hypotheses side by side.